In [22]:
import pandas as pd
import numpy as np
from jupyter_utils import jupyter_setup, load_tracker
import ipywidgets as widgets
jupyter_setup()
import plotly.graph_objects as go
import plotly.express as ex
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

D:\Archiv\Studium\Master\6.-Semester\Masters_Thesis\Git\acoustic_covid_detection\python


In [23]:
tracker = load_tracker()
# tracker.summarize(detail="compact")

Loading Tracker from: D:/Archiv/Studium/Master/6.-Semester/Masters_Thesis/Git/acoustic_covid_detection/python/run/tracker_saves/2023-03-20_resnet18_logmel_combined_breaths_6s_FFT2048_fmax5500_112logmel_1_ChannelResnetInput_weightsOfFirstChannel_hyperparams_0v1.pickle

Parameters used for finding the best performing epoch:
           metric_used:                   auc_roc
           smoothing:                     5
           ignore_first_n_epochs:         5


D:\Archiv\Studium\Master\6.-Semester\Masters_Thesis\Git\acoustic_covid_detection\python\evaluation_and_tracking.py:259: RuntimeWarning:

Mean of empty slice

C:\Users\Michi\Anaconda3\envs\python_v3-8\lib\site-packages\numpy\lib\nanfunctions.py:1878: RuntimeWarning:

Degrees of freedom <= 0 for slice.



In [31]:
tracker.summarize(detail="compact")
# tracker.summarize(detail="detail")


Datasets used:
logmel_combined_breaths_6s_FFT2048_fmax5500_112logmel
           dataset_class:                 <class 'datasets.ResnetLogmelDataset'>
           participants_file:             2023_03_13_logmel_combined_breaths_6s_FFT2048_fmax5500_112logmel.pickle
           augmented_files:               ['2023_03_14_logmel_combined_breaths_3s_FFT2048_fmax5500_112logmelaugmented.pickle']
           train_set_label_counts:        label '0': 2266  -  label '1': 2196
           types_of_recording:            combined_breaths

Audio Processing Parameters:
           type_of_features:              logmel
           n_time_steps:                  224
           n_features:                    112
           sample_rate:                   22050
           n_fft:                         8192
           window_length:                 2048
           hop_size:                      1024
           fmin:                          0
           fmax:                          5512
           hop_size_

In [25]:
pd.DataFrame(tracker.augmentations)

,AddGaussianNoise,PitchShift,TimeStretch,Gain
0,"'probability': 0.8, 'min_amplitude': 0.0003, 'max_amplitude': 0.01","'probability': 0.8, 'min_semitones': -2, 'max_semitones': 2","'probability': 0.8, 'min_rate': 0.95, 'max_rate': 1.05","'probability': 0.8, 'min_gain_in_db': -18, 'max_gain_in_db': 12"


In [26]:
import pandas as pd
key = tracker.datasets.keys()
key=list(key)[0]
print(key)
print(tracker.model_and_training_parameters["compact"])
pd.DataFrame(tracker.datasets[key])

logmel_combined_breaths_6s_FFT2048_fmax5500_112logmel
{'model': 'ResNet', 'optimizer': 'Adam ', 'loss_function': 'BCEWithLogitsLoss'}


,dataset_class,participants_file,augmented_files,train_set_label_counts,types_of_recording
0,<class 'datasets.ResnetLogmelDataset'>,2023_03_13_logmel_combined_breaths_6s_FFT2048_fmax5500_112logmel.pickle,2023_03_14_logmel_combined_breaths_3s_FFT2048_fmax5500_112logmelaugmented.pickle,label '0': 2266 - label '1': 2196,combined_breaths


In [27]:
_ = widgets.interact(tracker.get_data_and_model_params, dataset_and_model=False, audio_processing=True, augmentations=False)

interactive(children=(Checkbox(value=False, description='dataset_and_model'), Checkbox(value=True, description…

# Compute Tracker Metrics:
when loading the tracker (load_tracker()), the recorded data will be loaded and preprocessed (cross validation mean/std and much more will be calculated). This is essential. Otherwise none of the following functionality works. You can call the preprocessing function manually (tracker.compute_overall_metrics()) to change the settings for processing. The available parameters are: <br>
- smooth_n_samples=10 <br>
This sets the number of samples over which a (weighted) moving average filter is applied before looking for a min/max of the curve or calculate the mean.
- ignore_first_n_epochs=5 <br>
Simply do not take the first n epochs into account when looking for a min/max
- metric_used_for_performance_analysis="auc_roc" <br>
The epoch on which the performance for all metrics will be evaluated for a specific run will be chosen by looking for the min/max in this metric. This means, that if the AUC ROC curve is chosen in the argument, it will look for the argmax of the AUC ROC curve and evaluate loss, accuracy, F1 etc in this exact epoch even when they have their extremum in another epoch  <br>

In [28]:
n_samples = 10
tracker.compute_overall_metrics(smooth_n_samples=n_samples, ignore_first_n_epochs=n_samples, metric_used_for_performance_analysis="auc_roc")


Parameters used for finding the best performing epoch:
           metric_used:                   auc_roc
           smoothing:                     10
           ignore_first_n_epochs:         10


D:\Archiv\Studium\Master\6.-Semester\Masters_Thesis\Git\acoustic_covid_detection\python\evaluation_and_tracking.py:259: RuntimeWarning:

Mean of empty slice

C:\Users\Michi\Anaconda3\envs\python_v3-8\lib\site-packages\numpy\lib\nanfunctions.py:1878: RuntimeWarning:

Degrees of freedom <= 0 for slice.



# Evaluate the Cross Validation Results
For each run within a cross validation setting, the epoch of the min/max will be calculated and all metrics evaluated at this metric (after being smoothed as specified). The mean and std are calculated from the (in the default case) epoch where AUC ROC hits the global maximum. The resulting table can be adjusted with the jupyter widgets: <br>
- The styled table can be sorted by metric values (first drop down)  <br>
- by ticking "show_full_df" you can also display e.g. the standard deviation of each metric
- If you tick "highlight parameters" all hyperparameters, that changed thoughout the different runs, are highlighted by value. <br>
- The sliders below can be adjusted to filter out all runs with results worse than the slider setting  <br>
For more details (standard deviation e.g.), the cell below does exaclty the same but with 

In [29]:
def sortby(sort_descending_by_metric, highlight_hyperparameters, show_full_df, **kwargs):
    # thresholds = {"auc_roc": auc_threshold}
    if show_full_df:
        df = tracker.full_metric_performance_df
    else:
        df = tracker.compact_metric_performance_df
    style = tracker.style_metric_performance_df(df, sort_by=sort_descending_by_metric, highlight_hyperparameters=highlight_hyperparameters, filter_thresholds=kwargs)
    return style
_ = widgets.interact(sortby, sort_descending_by_metric=tracker.metrics_used, highlight_hyperparameters=True, show_full_df=False,
                auc_roc=widgets.FloatSlider(min=0.0, max=1.0, value=0.6, step=0.005),
                accuracy=widgets.FloatSlider(min=0.0, max=1.0, value=0.6, step=0.005),
                tpr=widgets.FloatSlider(min=0.0, max=1.0, value=0.4, step=0.005),
                tnr=widgets.FloatSlider(min=0.0, max=1.0, value=0.6, step=0.005)
                )

interactive(children=(Dropdown(description='sort_descending_by_metric', options=('auc_roc', 'loss', 'accuracy'…

# Box Plot the Hyperpaprameter Performances
Each metric is viewed separately. You can also choose to only look at the n_best performing hyperparameter runs. <br>
Depending on whether "sort_by_current_metric" is ticked this means something different. If it is not ticked, all runs will be sorted by the metric defined in the beginning (tracker.compute_overall_metrics()). Otherwise it will be sorted by the performance of the metric you have chosen to look at. This is very usefull  to get insight on which setting of a specific hyperparameter affects specific metrics. Some hyperparameter may for example improve the AUC ROC a bit but in the F1 or tnr metric they clearly hava a negative impact.

In [9]:
# hyperparameters = (list(tracker.crossval_runs[0].parameters.keys()))
_ = widgets.interact(tracker.boxplot_run_statistics, metric=tracker.metrics_used,
                     show_n_best_runs=widgets.IntSlider(value=tracker.n_hyperparameter_runs, min=1, max=tracker.n_hyperparameter_runs),
                     color_by_hyperparameter=[None, *tracker.hyperparameters],
                     sort_by_current_metric=True)

interactive(children=(Dropdown(description='metric', options=('auc_roc', 'loss', 'accuracy', 'f1_score', 'auc_…

# Metric Performance over Epochs
Here we can analyze the cross validation mean of each hyperparameter run over the epochs. You can switch between train and eval mode and choose a specific metric to look at. The solid line is the mean over all folds within a crossval run. This is calculated epoch-wise, so at each epoch the smoothed metric performance of all folds is averaged. In the same way, the standard devaition is calculated and shown as area plot. <br>
You can choose to display the area plot (std) and also to view the fold within each crossval run + the epoch in which the in/max of the specified metric (in tracker.compute_overall_metrics()) was found. Additionally you can choose to view only the n best performing hyperparameter runs. The ranking is calculated by the metric set at the beginning (tracker.compute_overall_metrics()). <br>
You can change the number of samples to smooth the curves with. The default setting was also set in the compute_overall_metrics method. This is only for visualization and does not change the settings which are used to calculate the final crossval performance.<br>
Below is a similar method but it only shows 1 crossval run which can be switched with the dropdown menu.

In [10]:
tracker.crossval_runs[0].runs[4].best_performances["eval"]
# tracker.crossval_runs[0].runs.pop(1)

IndexError: list index out of range

In [11]:
_ = widgets.interact(tracker.show_all_runs, 
                     mode=["eval", "train"], 
                     metric=tracker.metrics_used, 
                     n_samples_for_smoothing=widgets.IntSlider(value=tracker.performance_eval_params["smoothing"], min=1, max=15), 
                     show_separate_folds=True, 
                     show_std_area_plot=False,
                     show_n_best_runs=widgets.IntSlider(value=10, min=1, max=tracker.n_hyperparameter_runs),
                     color_by_hyperparameter=[None, *tracker.hyperparameters])

interactive(children=(Dropdown(description='mode', options=('eval', 'train'), value='eval'), Dropdown(descript…

In [ ]:
def get_list_of_best_n_runs(sort_by_metric=True, metric="auc_roc"):
    # this way, the dropdown menu of run_id below is sorted by metric performance specified
    if sort_by_metric:
        hyperparameter_settings = [str(tracker.crossval_runs[idx].parameters) for idx in tracker.get_indices_of_best_n_runs(tracker.n_hyperparameter_runs, metric)]
    else:
        hyperparameter_settings = tracker.run_ids
    return hyperparameter_settings

In [ ]:
_ = widgets.interact(tracker.show_single_run, 
                 run_id=get_list_of_best_n_runs(sort_by_metric=True, metric="auc_roc"),
                 mode=["eval", "train"], 
                 metric=tracker.metrics_used, 
                 n_samples_for_smoothing=widgets.IntSlider(value=tracker.performance_eval_params["smoothing"], min=1, max=15), 
                 show_separate_folds=True, 
                 show_std_area_plot=True)